\begin{align}
    Q_k(N) &= \frac{1}{k!} \sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N} \delta_{0, \sum_{l=1}^{k-1}\sum_{m=l+1}^{k}\delta_{i_l, i_m} + \delta_{j_l, j_m} + \delta_{j_l-i_l,j_m-i_m} + \delta_{j_i+i_l, j_m+i_m}} \\
    &\stackrel{\spadesuit}{=} \frac{1}{(4N)! k!}\sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N}\sum_{s=0}^{4N}S_1(4N + 1, s + 1) \\ 
    &\cdot \left(\sum_{l=1}^{k-1}\sum_{m=l+1}^{k}\delta_{i_l, i_m} + \delta_{j_l, j_m} + \delta_{j_l-i_l,j_m-i_m} + \delta_{j_i+i_l, j_m+i_m}\right)^s \\
    &\stackrel{\clubsuit}{=} \frac{1}{(4N)!k!}\sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N}\sum_{s=0}^{4N}S_1(4N + 1, s + 1) \frac{1}{(2N)!^{2s}}\Biggl( \\
    &\sum_{l=1}^{k-1}\sum_{m=l+1}^{k} \sum_{t=0}^{2N} \left[\sum_{u=0}^{t}(-1)^{-t+u} S_1(2N+1, u+1) S_1(2N+1,t-u+1)\right] \\
    &\cdot \left[(i_l - i_m)^t + (j_l - j_m)^t + (i_l - i_m - j_l + j_m)^t + (i_l - i_m + j_l - j_m)^t\right] \Biggr)^s
\end{align}

In [7]:
import itertools

In [29]:
class S1_class:
    def __init__(self):
        self.cache = {}
        
    def _solve(self, n, k):
        if (n,k) in self.cache: return self.cache[(n,k)]
    
        # Base cases
        if n == k == 0:
            return 1
        if n == 0 or k == 0:
            return 0
        if k > n:
            return 0

        result = S1(n-1, k-1) - (n-1) * S1(n-1, k)
        self.cache[(n,k)] = result
        return result
    
    def __call__(self, n, k):
        return self._solve(n,k)

S1 = S1_class()

# Example usage
n, k = 5, 3
print(f"S1({n}, {k}) = {S1(n, k)}")

S1(5, 3) = 35


In [30]:
factorial = lambda n: 1 if n <= 1 else n * factorial(n-1)
factorial(5)

120

In [31]:
delta = lambda a,b: int(a == b)
delta(0,1), delta(0,0)

(0, 1)

\begin{align}
Q_k(N) &= \frac{1}{k!} \sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N} \delta_{0, \sum_{l=1}^{k-1}\sum_{m=l+1}^{k}\delta_{i_l, i_m} + \delta_{j_l, j_m} + \delta_{j_l-i_l,j_m-i_m} + \delta_{j_i+i_l, j_m+i_m}}
\end{align}

In [32]:
def Q_Nk_first(N, k):
    two_sum = list(itertools.product(*[range(1,N+1) for _ in range(2)]))
    total_sum = list(itertools.product(*[two_sum for _ in range(k)]))
    
    count = 0
    for combi in total_sum:
        # combi[a][0] = i_{a}, combi[a][1] = j_a
        count += delta(0, sum(
            sum(
                delta(combi[l-1][0], combi[m-1][0]) + delta(combi[l-1][1], combi[m-1][1]) + delta(combi[l-1][0] - combi[l-1][1], combi[m-1][0] - combi[m-1][1]) + delta(combi[l-1][0] + combi[l-1][1], combi[m-1][0] + combi[m-1][1]) 
                    for m in range(l + 1, k+1)
                ) 
                for l in range(1,k)
            )
        )
        
    return count/factorial(k)

Q_Nk_first(6,2)

340.0

\begin{align}
    Q_k(N) = \frac{1}{(4N)! k!}\sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N}\sum_{s=0}^{4N}S_1(4N + 1, s + 1) \cdot \left(\sum_{l=1}^{k-1}\sum_{m=l+1}^{k}\delta_{i_l, i_m} + \delta_{j_l, j_m} + \delta_{j_l-i_l,j_m-i_m} + \delta_{j_i+i_l, j_m+i_m}\right)^s
\end{align}

In [36]:
def Q_Nk_second(N, k):
    two_sum = list(itertools.product(*[range(1,N+1) for _ in range(2)]))
    total_sum = list(itertools.product(*[two_sum for _ in range(k)]))
    
    count = 0
    for combi in total_sum:
        # combi[a][0] = i_{a}, combi[a][1] = j_a
        count += sum(S1(4*N+1,s+1) * sum(
            sum(
                delta(combi[l-1][0], combi[m-1][0]) + delta(combi[l-1][1], combi[m-1][1]) + delta(combi[l-1][0] - combi[l-1][1], combi[m-1][0] - combi[m-1][1]) + delta(combi[l-1][0] + combi[l-1][1], combi[m-1][0] + combi[m-1][1]) 
                    for m in range(l + 1, k+1)
                ) 
                for l in range(1,k)
            )**s for s in range(4*N+1))
        
    return count/(factorial(k) * factorial(4*N))

Q_Nk_second(4,3)

24.0

\begin{align}
    Q_k(N) &= \frac{1}{(4N)!k!}\sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N}\sum_{s=0}^{4N}S_1(4N + 1, s + 1) \frac{1}{(2N)!^{2s}}\Biggl( \\
    &\sum_{l=1}^{k-1}\sum_{m=l+1}^{k} \sum_{t=0}^{2N} \left[\sum_{u=0}^{t}(-1)^{-t+u} S_1(2N+1, u+1) S_1(2N+1,t-u+1)\right] \\
    &\cdot \left[(i_l - i_m)^t + (j_l - j_m)^t + (i_l - i_m - j_l + j_m)^t + (i_l - i_m + j_l - j_m)^t\right] \Biggr)^s
\end{align}

\begin{align}
    &\delta_{a,b}^{(D)} = \frac{1}{(-1)^{D} D!^2}\prod_{k=1}^{D} ((b - a) - k)((b - a) + k) \\
    &= \frac{1}{(-1)^{D}D!^2} \sum_{s=0}^{2D} \left[\sum_{k=0}^{s}(-1)^{D-s+k} S_1(D+1, k+1) S_1(D+1,s-k+1)\right] (b - a)^s \label{eq:symmetric_stirling1}
\end{align}

In [157]:
def delta(a,b,D):
    # note that the 'int' is necessary to avoid overflow errors
    # Python can handle arbitrarily large integers, but not arbitrarily large floating points
    return sum(
        sum(int((-1)**(D-t+u)) * S1(D+1,u+1) * S1(D+1, t-u+1) for u in range(t+1)) * (b - a)**t
        for t in range(2*D+1)
    ) // ((-1)**D * factorial(D)**2)

for i in range(-8,12+1):
    print(i, delta(2,i,9))

-8 -92378
-7 0
-6 0
-5 0
-4 0
-3 0
-2 0
-1 0
0 0
1 0
2 1
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 -92378


In [159]:
def Q_Nk_third(N, k):
    two_sum = list(itertools.product(*[range(1,N+1) for _ in range(2)]))
    total_sum = list(itertools.product(*[two_sum for _ in range(k)]))
    
    count = 0
    for combi in total_sum:
        # combi[a][0] = i_{a}, combi[a][1] = j_a
        count += sum(S1(4*N+1,s+1) * sum(
            sum(
                delta(combi[l-1][0], combi[m-1][0],2*N) + delta(combi[l-1][1], combi[m-1][1],2*N) + delta(combi[l-1][0] - combi[l-1][1], combi[m-1][0] - combi[m-1][1],2*N) + delta(combi[l-1][0] + combi[l-1][1], combi[m-1][0] + combi[m-1][1],2*N) 
                    for m in range(l + 1, k+1)
                ) 
                for l in range(1,k)
            )**s for s in range(4*N+1))
        
    return count/(factorial(k) * factorial(4*N))

Q_Nk_third(4,2)

44.0

In [160]:
def Q_Nk_third(N, k):
    two_sum = list(itertools.product(*[range(1,N+1) for _ in range(2)]))
    total_sum = list(itertools.product(*[two_sum for _ in range(k)]))
    
    count = 0
    for combi in total_sum:
        # combi[a][0] = i_{a}, combi[a][1] = j_a
        count += sum(S1(4*N+1,s+1) * sum(
            sum(
                    # D = 2N
                    sum(
                        sum(int((-1)**(2*N-t+u)) * S1(2*N+1,u+1) * S1(2*N+1, t-u+1) for u in range(t+1)) * ((combi[m-1][0] - combi[l-1][0])**t + (combi[m-1][1] - combi[l-1][1])**t + (combi[m-1][0] - combi[m-1][1] - combi[l-1][0] + combi[l-1][1])**t + (combi[m-1][0] + combi[m-1][1] - combi[l-1][0] - combi[l-1][1])**t)
                        for t in range(2*2*N+1)
                    ) // (factorial(2*N)**2)
                
                for m in range(l + 1, k+1)
                ) 
                for l in range(1,k)
            )**s for s in range(4*N+1))
        
    return count//(factorial(k) * factorial(4*N))

Q_Nk_third(4,2)

44

In [161]:
# Moved the 1/(2N)!^2 out of the power s
def Q_Nk_third(N, k):
    two_sum = list(itertools.product(*[range(1,N+1) for _ in range(2)]))
    total_sum = list(itertools.product(*[two_sum for _ in range(k)]))
    
    count = 0
    for combi in total_sum:
        # combi[a][0] = i_{a}, combi[a][1] = j_a
        count += sum(S1(4*N+1,s+1) * sum(
            sum(
                    # D = 2N
                    sum(
                        sum(int((-1)**(2*N-t+u)) * S1(2*N+1,u+1) * S1(2*N+1, t-u+1) for u in range(t+1)) * ((combi[m-1][0] - combi[l-1][0])**t + (combi[m-1][1] - combi[l-1][1])**t + (combi[m-1][0] - combi[m-1][1] - combi[l-1][0] + combi[l-1][1])**t + (combi[m-1][0] + combi[m-1][1] - combi[l-1][0] - combi[l-1][1])**t)
                        for t in range(2*2*N+1)
                    )
                
                for m in range(l + 1, k+1)
                ) 
                for l in range(1,k)
            )**s  // (factorial(2*N)**(2*s)) for s in range(4*N+1))
        
    return count//(factorial(k) * factorial(4*N))

Q_Nk_third(4,2)

44

In [163]:
Q_Nk_third(4,3)

24

In [164]:
# Takes a while
Q_Nk_third(5,3) # expects 204

204

\begin{align}
Q_k(N) &= \frac{1}{(4N)!k!}\sum_{i_1=1}^{N}\sum_{j_1=1}^{N} \dots \sum_{i_k=1}^{N}\sum_{j_k=1}^{N}\sum_{s=0}^{4N}S_1(4N + 1, s + 1) \frac{1}{(2N)!^{2s}}\Biggl( \\
    &\sum_{l=1}^{k-1}\sum_{m=l+1}^{k} \sum_{t=0}^{2N} \left[\sum_{u=0}^{t}(-1)^{-t+u} S_1(2N+1, u+1) S_1(2N+1,t-u+1)\right] \\
    &\cdot \Biggl[\sum_{r=0}^{t}\binom{t}{r}(-1)^{t-r} (i_l^r i_m^{t-r} + j_l^rj_m^{t-r}) \\
    &+ \sum_{r_1 + r_2 + r_3 + r_4 = t}^{t}\binom{t}{r_1, r_2, r_3, r_4}\left[(-1)^{r_2 + r_3}  + (-1)^{r_2 + r_4}\right]i_l^{r_1}i_m^{r_2}j_l^{r_3}j_m^{r_4}\Biggr] \Biggr)^s
\end{align}

In [167]:
factorial = lambda n: 1 if n <= 1 else n * factorial(n-1)
factorial(5)

120

In [168]:
def multinomial_coefficient(n, counts):
    if sum(counts) != n:
        raise ValueError("The sum of group sizes must equal n.")
    
    numerator = factorial(n)
    denominator = 1
    for count in counts:
        denominator *= factorial(count)
    
    return numerator // denominator
multinomial_coefficient(5,[2,3])

10

In [ ]:
# Moved the 1/(2N)!^2 out of the power s
def Q_Nk_fourth(N, k):
    two_sum = list(itertools.product(*[range(1,N+1) for _ in range(2)]))
    total_sum = list(itertools.product(*[two_sum for _ in range(k)]))
    
    count = 0
    for combi in total_sum:
        # combi[a][0] = i_{a}, combi[a][1] = j_a
        count += sum(S1(4*N+1,s+1) * sum(
            sum(
                    # D = 2N
                    sum(
                        sum(int((-1)**(2*N-t+u)) * S1(2*N+1,u+1) * S1(2*N+1, t-u+1) for u in range(t+1)) * (
                            
                            sum(multinomial_coefficient(t, [r,t-r]) * int(-1)**(t-r) * ()) +
                            sum() # TODO
                            
                        )
                        for t in range(2*2*N+1)
                    )
                
                for m in range(l + 1, k+1)
                ) 
                for l in range(1,k)
            )**s  // (factorial(2*N)**(2*s)) for s in range(4*N+1))
        
    return count//(factorial(k) * factorial(4*N))

Q_Nk_third(4,2)